# Wetlands LLM + MCP Testing

Test the LLM with MCP server integration using LangChain

In [30]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.tools import tool

## Configuration

Using remote MCP server at nrp-nautilus.io

In [31]:
# Load config
with open('../maplibre/config.json') as f:
    config = json.load(f)

# Get API key from environment
api_key = os.getenv('NRP_API_KEY')
if not api_key:
    print("WARNING: NRP_API_KEY not set!")
    api_key = input("Enter your NRP API key: ")

# Remote MCP server URL (hosted on nrp-nautilus.io)
mcp_url = config["mcp_server_url"]
llm_endpoint = config['llm_host']  # Use llm_host to leverage api_key and bypass proxy
llm_model = "glm-4.6" # config['llm_model']

print(f"LLM Endpoint: {llm_endpoint}")
print(f"Model: {llm_model}")
print(f"MCP Server: {mcp_url}")

LLM Endpoint: https://ellm.nrp-nautilus.io/v1
Model: glm-4.6
MCP Server: https://biodiversity-mcp.nrp-nautilus.io/sse


## Load System Prompt

In [32]:
with open('../maplibre/system-prompt.md') as f:
    system_prompt = f.read()

print(f"System prompt loaded: {len(system_prompt)} characters")

System prompt loaded: 6185 characters


## Setup MCP Integration with SSE Transport

Connect to the remote MCP server using SSE (Server-Sent Events) transport:

In [33]:
from langchain_mcp_adapters.client import MultiServerMCPClient

# Connect to the remote MCP server's SSE endpoint
client = MultiServerMCPClient(
    {
        "wetlands": {
            "transport": "sse",
            "url": mcp_url,
        }
    }
)

# Get tools from the MCP server
tools = await client.get_tools()

print(f"✓ Connected to remote MCP server!")
print(f"✓ Available tools: {[tool.name for tool in tools]}")

✓ Connected to remote MCP server!
✓ Available tools: ['query']


## Initialize LLM with MCP Tools

In [34]:
# Initialize LangChain LLM with MCP tools
llm = ChatOpenAI(
    base_url=llm_endpoint,
    api_key=api_key,
    model=llm_model,
    temperature=0.7
)

# Bind the MCP tools to the LLM
llm_with_tools = llm.bind_tools(tools)

print(f"LLM initialized with {len(tools)} MCP tools")

LLM initialized with 1 MCP tools


## Test LLM with System Prompt (No Tools)

Test that the LLM responds correctly with the system prompt but without using any tools:

In [35]:
# Test LLM without tools, just with system prompt
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="What are some types of wetland?")
]

response = llm.invoke(messages)
print("LLM Response (without tools):")
print(response.content)

LLM Response (without tools):

Based on the global wetlands dataset, wetlands are classified into several major categories with 33 specific types. Here are the main wetland categories and their types:

## **Open Water** (Codes 1-5)
- Freshwater lake
- Saline lake  
- Reservoir
- Large river
- Small river

## **Palustrine Wetlands** (Codes 13-15)
- Freshwater marsh - herbaceous wetlands with standing water
- Swamp forest - forested wetlands with standing water
- Flooded forest - seasonally inundated forests

## **Riverine Wetlands** (Codes 16-18)
- Floodplain - areas adjacent to rivers that flood periodically
- Oxbow lake - curved lakes formed from abandoned river channels
- Riverine wetland - wetlands directly connected to river systems

## **Lacustrine Wetlands** (Codes 19-20)
- Lacustrine fringe - wetlands around lake edges
- Lacustrine marsh - marshy areas within lake systems

## **Ephemeral Wetlands** (Codes 21-23)
- Pan - shallow depressions that fill with water seasonally
- Inter

## Example: LLM with MCP Tool Calling

Now let's use the LLM to automatically generate SQL queries and call the MCP tools:

In [36]:
async def ask_wetlands_question(question: str):
    """Ask a natural language question about wetlands data"""
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=question)
    ]
    
    response = await llm_with_tools.ainvoke(messages)
    
    print(f"\n{'='*70}")
    print(f"❓ {question}")
    print(f"{'='*70}\n")
    
    # Debug: Show what the LLM returned
    print(f"🔍 DEBUG: response.tool_calls = {response.tool_calls}")
    print(f"🔍 DEBUG: response.content = {response.content[:200] if response.content else 'None'}...")
    print()
    
    # Handle tool calls
    if response.tool_calls:
        for tool_call in response.tool_calls:
            print(f"🔧 Tool call detected: {tool_call['name']}")
            print(f"🔍 DEBUG: args = {tool_call['args']}\n")
            
            # Check if args contains the query
            if 'query' not in tool_call['args'] or not tool_call['args']['query']:
                print(f"⚠️  WARNING: Tool call missing 'query' argument!")
                print(f"   The model may be confused. Trying to extract query from content...")
                
                # Try to help by asking the model to provide the query explicitly
                retry_messages = [
                    SystemMessage(content=system_prompt),
                    HumanMessage(content=question),
                    HumanMessage(content="Please provide the exact SQL query to answer this question. Use the 'query' tool with the SQL as the argument.")
                ]
                retry_response = await llm_with_tools.ainvoke(retry_messages)
                
                if retry_response.tool_calls and retry_response.tool_calls[0]['args'].get('query'):
                    tool_call = retry_response.tool_calls[0]
                    print(f"✅ Retry successful, got query!")
                else:
                    print(f"❌ Retry failed. Skipping this query.")
                    return None
            
            print(f"🔧 Executing SQL query...\n")
            print(tool_call['args']['query'])
            print("-" * 70)
            
            # Execute the tool with a fresh client connection
            try:
                # Create a fresh client for each tool invocation to avoid session timeouts
                fresh_client = MultiServerMCPClient(
                    {
                        "wetlands": {
                            "transport": "sse",
                            "url": mcp_url,
                        }
                    }
                )
                fresh_tools = await fresh_client.get_tools()
                fresh_tool = next((t for t in fresh_tools if t.name == tool_call['name']), None)
                
                if fresh_tool:
                    result = await fresh_tool.ainvoke(tool_call['args'])
                    
                    print(f"\n📊 Raw result:")
                    print(result)
                    print("-" * 70)
                    
                    # Get LLM interpretation
                    messages.append(response)
                    messages.append(HumanMessage(content=f"Tool result: {result}"))
                    final = await llm.ainvoke(messages)
                    
                    print(f"\n💬 Answer:")
                    print(final.content)
                    return final.content
                else:
                    print(f"❌ Tool '{tool_call['name']}' not found")
                    return None
            except Exception as e:
                print(f"❌ Error: {e}")
                import traceback
                traceback.print_exc()
                return None
    else:
        print(f"💬 {response.content}")
        return response.content


## Example Questions

Ask natural language questions - the LLM will generate SQL and use the MCP tools automatically:

In [37]:
await ask_wetlands_question("How many hectares of wetlands are there in total?")


❓ How many hectares of wetlands are there in total?

🔍 DEBUG: response.tool_calls = [{'name': 'query', 'args': {'query': "-- Set threads for parallel I/O (S3 reads are I/O bound, use more threads than cores)\nSET THREADS=100;\n\n-- Install and load httpfs extension for S3 access\nINSTALL httpfs;\nLOAD httpfs;\n\n-- Configure S3 connection to MinIO\nCREATE OR REPLACE SECRET s3 (\n    TYPE S3,\n    ENDPOINT 'minio.carlboettiger.info',\n    URL_STYLE 'path'\n);\n\n-- Calculate total wetland area globally\nSELECT \n    COUNT(DISTINCT h8) as total_hexagons,\n    ROUND(COUNT(DISTINCT h8) * 73.7327598, 2) as total_hectares,\n    ROUND(COUNT(DISTINCT h8) * 0.737327598, 2) as total_km2,\n    ROUND(COUNT(DISTINCT h8) * 0.284679, 2) as total_sq_miles\nFROM read_parquet('s3://public-wetlands/hex/**')\nWHERE Z > 0;"}, 'id': 'chatcmpl-tool-4896f3a5c7e44c3a89ed7be3d7f2f170', 'type': 'tool_call'}]
🔍 DEBUG: response.content = 
I'll calculate the total global wetland area by querying the wetlands datas

"\nBased on the global wetlands data, there are **5.59 billion hectares** of wetlands worldwide.\n\n**Total Global Wetland Area:**\n- **5,586,046,970 hectares** (5.59 billion hectares)\n- **55,860,470 square kilometers**\n- **21,567,486 square miles**\n\nThis represents **75,760,720 individual hexagonal cells** across the globe, each covering approximately 73.7 hectares.\n\n**Geographic Context:**\n- This area is equivalent to about **3.8% of Earth's total land surface**\n- It's larger than the entire continent of South America (~17.8 million km²)\n- It's roughly **3.4 times the size of the United States** (~9.8 million km²)\n- It's approximately **1.5 times the size of the Antarctic continent** (~14 million km²)\n\nThis comprehensive dataset includes all wetland types, from open water bodies and coastal systems to inland peatlands, marshes, and floodplain forests. The scale highlights the critical importance of wetlands as one of Earth's most extensive ecosystem types.\n\nWould you li

In [38]:
await ask_wetlands_question("Compute the total area of each wetland code, sort largest first")


❓ Compute the total area of each wetland code, sort largest first

🔍 DEBUG: response.tool_calls = [{'name': 'query', 'args': {'query': "-- Set threads for parallel I/O (S3 reads are I/O bound, use more threads than cores)\nSET THREADS=100;\n\n-- Install and load httpfs extension for S3 access\nINSTALL httpfs;\nLOAD httpfs;\n\n-- Configure S3 connection to MinIO\nCREATE OR REPLACE SECRET s3 (\n    TYPE S3,\n    ENDPOINT 'minio.carlboettiger.info',\n    URL_STYLE 'path'\n);\n\nSELECT \n    Z as wetland_code,\n    CASE \n        WHEN Z = 1 THEN 'Freshwater lake'\n        WHEN Z = 2 THEN 'Saline lake'\n        WHEN Z = 3 THEN 'Reservoir'\n        WHEN Z = 4 THEN 'Large river'\n        WHEN Z = 5 THEN 'Small river'\n        WHEN Z = 6 THEN 'Coastal lagoon'\n        WHEN Z = 7 THEN 'Delta'\n        WHEN Z = 8 THEN 'Estuary'\n        WHEN Z = 9 THEN 'Reef'\n        WHEN Z = 10 THEN 'Salt marsh'\n        WHEN Z = 11 THEN 'Mangrove'\n        WHEN Z = 12 THEN 'Coastal wetland'\n        WHEN Z =

Traceback (most recent call last):
  File "/tmp/ipykernel_1322360/3323704588.py", line 64, in ask_wetlands_question
    result = await fresh_tool.ainvoke(tool_call['args'])
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cboettig/Documents/github/boettiger-lab/wetlands/.venv/lib/python3.12/site-packages/langchain_core/tools/structured.py", line 66, in ainvoke
    return await super().ainvoke(input, config, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cboettig/Documents/github/boettiger-lab/wetlands/.venv/lib/python3.12/site-packages/langchain_core/tools/base.py", line 615, in ainvoke
    return await self.arun(tool_input, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cboettig/Documents/github/boettiger-lab/wetlands/.venv/lib/python3.12/site-packages/langchain_core/tools/base.py", line 1061, in arun
    raise error_to_raise
  File "/home/cboettig/Documents/github/boettiger-lab/wetlands/.venv/lib/pytho

In [22]:
# Example 2: Peatlands
await ask_wetlands_question("What is the total area of peatlands in square kilometers?")


❓ What is the total area of peatlands in square kilometers?

🔍 DEBUG: response.tool_calls = [{'name': 'query', 'args': {'query': "SET THREADS=100; INSTALL httpfs; LOAD httpfs; CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path'); SELECT COUNT(DISTINCT h8) * 0.737327598 as total_km2 FROM read_parquet('s3://public-wetlands/hex/**') WHERE Z BETWEEN 24 AND 31;"}, 'id': 'chatcmpl-tool-5453d0d1f1f040ceb1a97b5b158bbde0', 'type': 'tool_call'}]
🔍 DEBUG: response.content = 
I'll calculate the total area of peatlands globally using the wetlands dataset. Peatlands are represented by codes 24-31 in the Z column.
...

🔧 Generating and executing SQL query...

SET THREADS=100; INSTALL httpfs; LOAD httpfs; CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path'); SELECT COUNT(DISTINCT h8) * 0.737327598 as total_km2 FROM read_parquet('s3://public-wetlands/hex/**') WHERE Z BETWEEN 24 AND 31;
-------------------------------------

"\nBased on the analysis of the global wetlands dataset, the total area of peatlands worldwide is approximately **13,891,247 square kilometers**.\n\nThis represents a significant portion of the Earth's wetland ecosystems, with peatlands (which include bogs, fens, mires, and other peat-forming wetlands) covering nearly 14 million square kilometers globally.\n\nTo put this in perspective:\n- This area is roughly equivalent to the size of Africa (about 30.3 million km²), meaning peatlands cover nearly half of Africa's land area\n- Peatlands represent one of the largest carbon stores on Earth, containing twice as much carbon as all forest biomass\n- These ecosystems are crucial for biodiversity, water regulation, and climate stability\n\nThe calculation was based on counting all H3 hexagons (resolution 8) classified as peatlands (codes 24-31) and converting the count to square kilometers using the standard H3 area conversion factor."

In [ ]:
# Example 3: Top wetland types
await ask_wetlands_question("What are the top 5 most common wetland types by area?")

## Direct Tool Test

Test an MCP tool directly (without the LLM generating the query):

In [ ]:
# Find the query tool
query_tool = next((t for t in tools if t.name == "query"), None)

if query_tool:
    # Test it with a simple query
    test_query = """
    SET THREADS=100;
    INSTALL httpfs; LOAD httpfs;
    CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path');
    SELECT COUNT(DISTINCT h8) as total_hexagons 
    FROM read_parquet('s3://public-wetlands/hex/**')
    WHERE Z > 0
    LIMIT 10;
    """
    
    # Create a fresh client connection before invoking
    fresh_client = MultiServerMCPClient(
        {
            "wetlands": {
                "transport": "sse",
                "url": mcp_url,
            }
        }
    )
    fresh_tools = await fresh_client.get_tools()
    fresh_query_tool = next((t for t in fresh_tools if t.name == "query"), None)
    
    result = await fresh_query_tool.ainvoke({"query": test_query})
    print("Result:")
    print(result)
else:
    print("Query tool not found!")